In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler  # for feature scaling
from sklearn.model_selection import StratifiedGroupKFold  # for train/test split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.utils import compute_class_weight

In [ ]:
columns = ['patient_id', 
'avg_right_diff_eigen',
'avg_diff_sink_idx',
'avg_diff_source_infl',
'avg_diff_sink_conn',
'class',
'label']

Create a Pandas dataframe of these columns for the following model. We named it success_df for Engel 1 and failure_df for Engels 2-4, then combined into one dataframe

In [ ]:
x = np.array(model_df.iloc[:,[1,2,3,4]])
y = np.array(model_df['label'])
ids = model_df['patient_id']

In [ ]:
y_test_pred_optimal_all = np.zeros((int(model_df.shape[0]),3))
y_test_all = np.zeros((int(model_df.shape[0]),3))
y_test_pred_all = np.zeros((int(model_df.shape[0]),3))

num_iterations = 10
optimal_thresholds_all = np.zeros((num_iterations,3))
num_classes = 2

sss = StratifiedGroupKFold(n_splits=10) #10Fold Cross Validation
for fold_idx, (train_index, test_index) in enumerate(sss.split(x, y, ids)):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        test_df = model_df.iloc[test_index]

        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        #Fit training data
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weight_dict = dict(zip(np.unique(y_train), class_weights))
        logreg = LogisticRegression(max_iter = 10000, class_weight=class_weight_dict)
        logreg.fit(x_train, y_train)

        #Apply fit to the predicted fold
        y_pred = logreg.predict(x_train)
        y_prob = logreg.predict_proba(x_train)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_train, y_prob)

        #Find optimal threshold
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        optimal_thresholds_all[fold_idx] = optimal_threshold

        #Testing
        y_test_pred = logreg.predict_proba(x_test)[:, 1]
        y_test_pred_optimal = (y_test_pred >= optimal_threshold).astype(int)

